In [30]:
#Imports
from scripts import dataset_scripts as ds_s, mauve_quantization as mq, subset_selection as ss
from transformers import AutoTokenizer, AutoModel
import torch
from datasets import Dataset, load_from_disk
import numpy as np

In [2]:
# Dicts are in format: {ds_path, ds_name, under_ds_name}
#human_ds_dict = {"ds_path":"data/human/news-fi-2019.jsonl", "ds_name":"news-fi-2019.jsonl", "under_ds_name":None}
#clums_ds_dict = {"ds_path":"data/clumsified/news-fi-2019.jsonl_regeneration_5_mini_regen_round_1.jsonl", "ds_name":"news-fi-2019.jsonl_regeneration_5_mini_regen_round_1.jsonl", "under_ds_name":"news-fi-2019.jsonl"}


#ds = ds_s.format_datasets([human_ds_dict, clums_ds_dict])
#ref, remaining = ds_s.sample_reference_corpus(ds, "news-fi-2019.jsonl", 5000)

In [3]:
model_name = "intfloat/multilingual-e5-large-instruct"
import os
os.environ['HF_HOME'] = '/scratch/project_2000539/tapio/HF_cache/'

In [4]:
#Model name that can be loaded from HF goes here
model_id = model_name
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, device_map="auto")


# ## Getting the embeddings with the wanted (L)LM

#inputs_q = [tokenizer.encode(x['text'], return_tensors="pt", truncation=True, max_length=512) for x in remaining]
#embeddings_q = mq.featurize_tokens_from_model(model, inputs_q, 1, name="", verbose=False)
#inputs_q = []
#del inputs_q
#inputs_p = [tokenizer.encode(x['text'], return_tensors="pt", truncation=True, max_length=512) for x in ref]
#embeddings_p = mq.featurize_tokens_from_model(model, inputs_p, 1, name="", verbose=False)
#inputs_p = []
#del inputs_p

In [5]:
#for i in range(len(ref)):
#    ref[i]['embedding']= embeddings_p[i]

#for i in range(len(remaining)):
#    remaining[i]['embedding']= embeddings_q[i]


#torch.save(ref, 'data/embs/ref.pt')

#ds_test = Dataset.from_list(ref)
#ds_test.save_to_disk("data/embs/ref.hf")

#ds_test = Dataset.from_list(remaining)
#ds_test.save_to_disk("data/embs/remaining.hf")

#torch.load(remaining, 'data/embs/remaining.pt')

In [6]:
ref_ds = load_from_disk("data/embs/ref.hf")
remaining_ds = load_from_disk("data/embs/remaining.hf")

In [7]:
ref = ref_ds.to_list()
remaining = remaining_ds.to_list()

In [8]:
embeddings_p = [torch.tensor(x['embedding']) for x in ref]
embeddings_q = [torch.tensor(x['embedding']) for x in remaining]

In [9]:
#Estimate the optimal number of clusters as done in MAUVE
num_of_clusters = max(2, int(round(min(len(embeddings_p)/10, len(embeddings_q)/10))))

print(f'The number of clusters is {num_of_clusters}')
results = mq.CDOE(torch.cat(embeddings_p), torch.cat(embeddings_q), num_of_clusters, verbose=True)

The number of clusters is 500
seed = 0
performing clustering in lower dimension = 392
'Shape of embeddings before PCA transformation (43030, 1024)'
'Shape of embeddings after PCA transformation (43030, 393)'
Clustering 43030 points in 393D to 500 clusters, redo 5 times, 500 iterations
  Preprocessing in 0.01 s
Outer iteration 0 / 5
  Iteration 499 (63.13 s, search 62.07 s): objective=1.38885e+07 imbalance=1.760 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 499 (127.06 s, search 125.00 s): objective=1.38864e+07 imbalance=1.772 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 5
  Iteration 499 (190.51 s, search 187.42 s): objective=1.39157e+07 imbalance=1.902 nsplit=0       
Outer iteration 3 / 5
  Iteration 499 (253.96 s, search 249.84 s): objective=1.39172e+07 imbalance=1.832 nsplit=0       
Outer iteration 4 / 5
  Iteration 499 (317.29 s, search 312.15 s): objective=1.39139e+07 imbalance=1.848 nsplit=0       
kmeans ti

In [13]:
# Combining information to make future work easier

p2cluster = results['p2cluster']
for i in p2cluster:
    ref[i]['cluster_id'] = p2cluster[i]
q2cluster = results['q2cluster']
for i in q2cluster:
    remaining[i]['cluster_id'] = q2cluster[i]


In [14]:
from scripts import dataset_scripts as ds_s, mauve_quantization as mq, subset_selection as ss
p_distr = results['p_bin_counts']
q_distr = results['q_bin_counts']

In [31]:
q_human = [x for x in remaining if not x['under_collection']]
q_clums = [x for x in remaining if x['under_collection']]

In [27]:
test_simple = ss.get_target_n_per_cluster(p_distr, q_distr, 5000, True)
print(np.sum(list(test_simple.values())))

5000


In [29]:
test_complex = ss.get_target_n_per_cluster(p_distr, q_distr, 5000)
print(np.sum(list(test_complex.values())))

5000


In [48]:
# Writing the function to do a random subset that matches the 'target distribution'
import random
def random_subset_sampler(target_distribution, q2cluster):
    cluster2q = {}
    for key, value in q2cluster.items():
        if value in cluster2q:
            cluster2q[value].append(key)
        else:
            cluster2q[value] = [key]

    returnable = []
    for c in target_distribution:
        if target_distribution[c] > 0:
            returnable += random.sample(cluster2q[c], target_distribution[c])
    return returnable

test_sample = random_subset_sampler(test_simple, q2cluster)

In [49]:
test_sample_unders = [remaining[i]['under_collection_id'] for i in test_sample]

In [50]:
print(f'Ratio of human texts in subset: {len([x for x in test_sample_unders if x])/len(test_sample_unders)}')

Ratio of human texts in subset: 0.4972


In [52]:
print(results['p_bin_counts'])

[16  7 18 14 11 17 12 12  0  1 12  1  9  8 24  1  4 19  2 20  2 19  0  0
  7  5  6  2  3  5 24 14  1  2  0  3  7  1 21  4 11  6 10 30  1 34 10  5
 17  1 11 14 17 16  3  1  7 46  1  0 19  6 18 16  3  0 48  5 23  0  9 14
 18 22  0 14 11  1 11 10 24 15  8  5 13 13  4 10  0  0  1 26 10  3 12 10
 14 10  3  5  8 20 15 20 18  8 20  0 32  6 19  6  0 16  9  5 15  2  6  2
 20  0 11  3 13 27  1 13  4  7 17 10 13  2 29 10  8  7 16 10  5 11 20 29
 13  9 10  4  1  0 25  5  4  5  2  2  4  2 11 27  0 22 12  0  1 12 14  4
 20 15  0 17  2 28  0  2  2 16 10 12  1  0  5  6  1 36 11  4 30  1  7  2
  0  8 10  2 14  1  8  2  0  7  7 18 13  1 13  0  1  0 13  6  0  6  1  3
 15 13  1  8 21  9  9 18 22  7 29  2 25  1  9  7  6 11 21  7  8 16  0  7
  9  1  8  5 10  1 22  0 30  8 19 13  0  1  0  1  7 18  2 34 13  4  6 21
  7 16 26 11  7  0  2 25  0 30  0 14  3 51  0  5  6 18  0  0  1 22 16  0
  3  0  0  0 19  3 31 15 37  7  4  5 17  9 23 22 22 17 16 16  0 11  7  1
 12 37 11 55  1  0 13  0  1 32  6  2  0 12  0 22  2

In [55]:
print(np.array(list(test_simple.values())))

[16  7 18 14 11 17 12 12  0  1 12  1  9  8 24  1  4 19  2 20  2 19  0  0
  7  5  6  2  3  5 24 14  0  2  0  3  7  1 21  4 11  6 10 30  1 34 10  5
 17  1 11 14 17 16  3  1  7 46  1  0 19  6 18 16  3  0 48  5 23  0  9 14
 18 22  0 14 11  1 11 10 24 15  8  5 13 13  4 10  0  0  1 26 10  3 12 10
 14 10  3  5  8 20 15 20 18  8 20  0 32  6 19  6  0 16  9  5 15  2  6  2
 20  0 11  3 13 27  1 13  4  7 17 10 13  2 29 10  8  7 16 10  5 11 20 29
 13  9 10  4  1  0 25  5  4  5  2  2  4  2 11 27  0 22 12  0  1 12 14  4
 20 15  0 17  2 28  0  2  2 16 10 12  1  0  5  6  1 36 11  4 30  1  7  2
  0  8 10  2 14  1  8  2  0  7  7 18 13  1 13  0  1  0 13  6  0  6  1  3
 15 13  1  8 21  9  9 18 22  7 29  2 25  1  9  7  6 11 21  7  8 16  0  7
  9  1  8  5 10  1 22  0 30  8 19 13  0  1  0  1  7 18  2 34 13  4  6 21
  7 16 26 11  7  0  2 25  0 30  0 14  3 52  0  5  6 18  0  0  1 22 16  0
  3  0  0  0 19  3 31 15 37  7  4  5 17  9 23 22 22 17 16 16  0 11  7  1
 12 37 11 55  1  0 13  0  1 32  6  2  0 12  0 22  2